In [49]:
#https://www.kaggle.com/code/purvitsharma/amazon-reviews-bidirectional-lstm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
#%matplotlib inline
plt.style.use("fivethirtyeight")
import seaborn as sns
sns.set_style("darkgrid")
#from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score
import re, string, nltk
#import emoji, bz2
#import emoji
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
#import keras
#import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, Dropout
from keras.layers import Embedding
#from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
import warnings
warnings.filterwarnings("ignore")
import pickle

In [18]:

product_training = pd.read_json("devided_dataset_v2/Grocery_and_Gourmet_Food/train/product_training.json")
review_training = pd.read_json("devided_dataset_v2/Grocery_and_Gourmet_Food/train/review_training.json")

product_training = product_training[0:10000]
review_training = review_training[0:10000]

# print(review_training['reviewText'])
#print(review_training)
#print(product_training)

In [19]:
#review_training = review_training.dropna()
review_training1 = review_training[~review_training['reviewText'].isnull()]
#eview_training1 = review_training
print(review_training1)

                                  asin                        reviewerID  \
0     71F1F9B34E46A7F11E8B72C26CA861B6  D0A66D9D59DFB8808DE637FA50025550   
1     1755243A708579B595D1AAAFBADD7725  8D0A76D2383DCFC2B0DA3189FCC53490   
2     982EB9BA1A12C03C3635DCD2A6CFD79E  CEDFFFFA048A21A94B2272349949BD08   
3     08D86545D56A7C392B81B96F73DA85C0  8686C793EA21D71A574F18A847A733AC   
4     9A8BE2D7907DA443B9A72E712290C0B2  DE9971A7DF0055FCC6BF78AA387AEDA6   
...                                ...                               ...   
9995  5BEA67880D35E5C94B9A9BF720C29E9F  C51CE63A071D786E3F5BE5A79A371710   
9996  1053A7C830A62D204464573A0FC31E5E  364ECD45C16694CDB964B4BDB9371332   
9997  E7D59B55E92072E3919364F8DC2FC214  F7B3C9638D5A7F9CB830AD21E24C650D   
9998  98E3ECADA7EFF0BCCD14E86CC911BE71  1324780ADB9EF5DF8439055C6EA65983   
9999  89CE0ACE3A66D13B27D6FE3ACC4BE57A  4A382DA4950421DB11838CFE5BEA0E49   

      unixReviewTime  vote  verified   reviewTime  \
0         1438560000  None      Tr

In [38]:
print(review_training1)
#print(review_training1['reviewText'])

                                  asin                        reviewerID  \
0     71F1F9B34E46A7F11E8B72C26CA861B6  D0A66D9D59DFB8808DE637FA50025550   
1     1755243A708579B595D1AAAFBADD7725  8D0A76D2383DCFC2B0DA3189FCC53490   
2     982EB9BA1A12C03C3635DCD2A6CFD79E  CEDFFFFA048A21A94B2272349949BD08   
3     08D86545D56A7C392B81B96F73DA85C0  8686C793EA21D71A574F18A847A733AC   
4     9A8BE2D7907DA443B9A72E712290C0B2  DE9971A7DF0055FCC6BF78AA387AEDA6   
...                                ...                               ...   
4995  135AE7982B3DA88FC2FFE2C2E01F459F  87987FC38EBC3AA041F73B07CF0B0619   
4996  2745501E4E72F413B5A4D57497FB103D  B9ECCF5C90192041883612EA8084C74E   
4997  544AD3792EAEDAB81BCC3852888229CC  89CCBF4312085987B3904C4CFA4EA661   
4998  E28B5A3AF6A4B81743EA7099B96FCE83  C698CC10E014E30617E9C2CE4E4C0EFD   
4999  F237E7AE3418C0DF1B715CCB16765203  C0DF2BE0169038C1333AC0D14A3CEC69   

      unixReviewTime  vote  verified   reviewTime  \
0         1438560000  None      Tr

In [20]:
def reshape_features(review_df, target_df):
        # Change dict to df
        #data_list = [(key, *values) for key, values in read_product.items()]
        # Convert the list of tuples to a DataFrame
        #feature_df = pd.DataFrame(data_list, columns=['asin', '0', '1'])
        feature_df = pd.DataFrame(review_df, columns=['asin', 'reviewText'])
        # Merge data frames
        final_df = pd.merge(feature_df, target_df, on='asin')
        final_df = pd.DataFrame(final_df, columns=['reviewText', 'awesomeness'])
        return final_df

review_df = review_training1
target_df =  product_training[['asin','awesomeness']]
review_training1 = reshape_features(review_df, target_df)
print(review_training1)

                                             reviewText  awesomeness
0     Almost impossible to open, but the tea is firs...            0
1     These were given as token gifts to my accounts...            0
2             Love these teas.  Would highly recommend.            0
3                                                  good            0
4     Arrived fresh. Delicious blend of spices.\nNON...            1
...                                                 ...          ...
2413  Red Velvet is a stunning tea - flavors that ar...            0
2414  Though I bought this as the purest Stevia I sa...            0
2415  Well, this surprised me.......I don't care for...            1
2416                                       Good quality            1
2417                                         Love these            1

[2418 rows x 2 columns]


In [50]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
#print(stopwords.words('english'))


#used later 
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/anishafujii/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/anishafujii/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/anishafujii/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/anishafujii/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [22]:


def clean_text(df, field):
    df[field] = df[field].str.replace(r"@"," at ")
    df[field] = df[field].str.replace("#[^a-zA-Z0-9_]+"," ")
    df[field] = df[field].str.replace(r"[^a-zA-Z(),\"'\n_]"," ")
    df[field] = df[field].str.replace(r"http\S+","")
    df[field] = df[field].str.lower()
    #print(df["reviewText"])
    print(df)
    return df

clean_text(review_training1,"reviewText")

                                             reviewText  awesomeness
0     almost impossible to open, but the tea is firs...            0
1     these were given as token gifts to my accounts...            0
2             love these teas   would highly recommend             0
3                                                  good            0
4     arrived fresh  delicious blend of spices \nnon...            1
...                                                 ...          ...
2413  red velvet is a stunning tea   flavors that ar...            0
2414  though i bought this as the purest stevia i sa...            0
2415  well, this surprised me       i don't care for...            1
2416                                       good quality            1
2417                                         love these            1

[2418 rows x 2 columns]


,reviewText,awesomeness
0,"almost impossible to open, but the tea is firs...",0
1,these were given as token gifts to my accounts...,0
2,love these teas would highly recommend,0
3,good,0
4,arrived fresh delicious blend of spices \nnon...,1
...,...,...
2413,red velvet is a stunning tea flavors that ar...,0
2414,though i bought this as the purest stevia i sa...,0
2415,"well, this surprised me i don't care for...",1
2416,good quality,1


In [23]:
# Applying Lemmmatizer to remove tenses from texts.
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub('[^a-zA-Z0-9]',' ',text)
    #text= re.sub(emoji.get_emoji_regexp(),"",text)
    text = [lemmatizer.lemmatize(word) for word in text.split() if not word in set(stopwords.words('english'))]
    text = ' '.join(text)
    return text

# review_df = review_training
# target_df =  product_training[['asin','awesomeness']]
# final_df = reshape_features(review_df, target_df)
#print(text['reviewText'])
#preprocess_text(text['reviewText'])
review_training1["reviewTextClean"] = review_training1["reviewText"].apply(preprocess_text)
print(review_training1.head(15))
#print(review_training1['reviewTextClean'])
#print(final_df)

                                           reviewText  awesomeness  \
0   almost impossible to open, but the tea is firs...            0   
1   these were given as token gifts to my accounts...            0   
2           love these teas   would highly recommend             0   
3                                                good            0   
4   arrived fresh  delicious blend of spices \nnon...            1   
5                                 my favorite coffee             1   
6   donut house light roast     my new favorite   ...            1   
7   to the greater public, this is a perfect coffe...            1   
8                          just don't like the taste             1   
9                                         my favorite            1   
10  this is my favorite matcha tea  i find it is s...            1   
11  not too strong   just a nice, warm cup of tea ...            1   
12                           very good    ) me gusta             1   
13  i'm not a strang

In [24]:
review_training = review_training1[["awesomeness",'reviewTextClean']]
print(review_training)

      awesomeness                                    reviewTextClean
0               0              almost impossible open tea first rate
1               0   given token gift account tin nice told tea great
2               0                    love tea would highly recommend
3               0                                               good
4               1  arrived fresh delicious blend spice non gmo pr...
...           ...                                                ...
2413            0  red velvet stunning tea flavor almost beyond d...
2414            0  though bought purest stevia saw amazon uncut p...
2415            1  well surprised care pearled barley love hull l...
2416            1                                       good quality
2417            1                                               love

[2418 rows x 2 columns]


In [121]:

# Stores products into a pickle file for easier training process
def store_data(data):
    # Store Product dict to speed up sentiment analysis for future runs
    with open('sentimentLSTM.pkl', 'wb') as fp:
        pickle.dump(data, fp)
        print('dictionary saved successfully to file')

store_data(review_training)

dictionary saved successfully to file


In [51]:
# Uses pickle file to recall features stored
def recall_data():
    # Recall stores sentiment analysis from file
    read_product = {}
    with open('sentimentLSTM.pkl', 'rb') as fp:
        read_product = pickle.load(fp)
        # print(read_product)
    return read_product

review_training = recall_data()

In [52]:
X_train, X_test, y_train, y_test = train_test_split(np.array(review_training["reviewTextClean"]),np.array(review_training["awesomeness"]), test_size=0.25,random_state=42)
print(X_train.shape)
print(X_test.shape)


(480263,)
(160088,)


In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf2 = TfidfVectorizer(use_idf=True, tokenizer=word_tokenize)
X_train_tf2 = tfidf2.fit_transform(X_train)
X_test_tf2 = tfidf2.transform(X_test)

In [63]:
# review_df = review_training
# target_df =  product_training[['asin','awesomeness']]
# final_df = reshape_features(review_df, target_df)

#review_training1.awesomeness.unique()

#X_train, X_test, y_train, y_test = train_test_split(np.array(review_training["reviewTextClean"]),np.array(review_training["awesomeness"]), test_size=0.25,random_state=42)
# print(X_train.shape)
# print(X_test.shape)


#from sklearn.feature_extraction.text import TfidfVectorizer

rf = RandomForestClassifier()
rf.fit(X_train_tf2, y_train)

#from sklearn.metrics import roc_auc_score
y_pred = rf.predict(X_test_tf2)
acc = accuracy_score(y_pred, y_test)
report = classification_report(y_test, y_pred)
roc = roc_auc_score(y_test,y_pred)
print(f"Accuracy: {acc*100}% and Roc Auc Score:{roc_auc_score(y_test,y_pred)}")
print(report)

Accuracy: 52.4390243902439% and Roc Auc Score:0.5208333333333333
              precision    recall  f1-score   support

           0       0.44      0.50      0.47        68
           1       0.60      0.54      0.57        96

    accuracy                           0.52       164
   macro avg       0.52      0.52      0.52       164
weighted avg       0.53      0.52      0.53       164



In [64]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train_tf2, y_train)

y_pred3 = nb.predict(X_test_tf2)
acc3 = accuracy_score(y_pred3, y_test)
report3 = classification_report(y_test, y_pred3)
roc3 = roc_auc_score(y_test,y_pred3)
print(f"Accuracy: {acc3*100}% and Roc Auc Score:{roc_auc_score(y_test,y_pred3)}")
print(report3)

Accuracy: 60.36585365853659% and Roc Auc Score:0.5585171568627451
              precision    recall  f1-score   support

           0       0.54      0.29      0.38        68
           1       0.62      0.82      0.71        96

    accuracy                           0.60       164
   macro avg       0.58      0.56      0.54       164
weighted avg       0.59      0.60      0.57       164



In [54]:
#Deep Learning 
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split


X = review_training["reviewTextClean"]
y = review_training.awesomeness
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
# print(X_train.shape)
# print(X_test.shape)

In [55]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [56]:
# using tokenizer to transform text messages into training and testing set
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_seq_padded = pad_sequences(X_train_seq, maxlen=64)
X_test_seq_padded = pad_sequences(X_test_seq, maxlen=64)

X_train_seq_padded[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,  283,  264,  644,   66,   28,  960,    2,    3, 1263,   25,
         82,  567,   87,   82, 1424,    3,    2,   87,   82], dtype=int32)

In [57]:
# construct model
BATCH_SIZE = 100

from keras.utils.vis_utils import plot_model
model = Sequential()
model.add(Embedding(len(tokenizer.index_word)+1,64))
model.add(Bidirectional(LSTM(100, dropout=0,recurrent_dropout=0)))
model.add(Dense(128, activation="relu"))
model.add(Dense(1,activation="sigmoid"))

model.compile("adam","binary_crossentropy",metrics=["accuracy"])
model.summary()
#plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

2023-05-28 19:46:52.726564: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-28 19:46:52.731526: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-28 19:46:52.733601: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, None, 64)          4232000   
                                                                 
 bidirectional_7 (Bidirectio  (None, 200)              132000    
 nal)                                                            
                                                                 
 dense_14 (Dense)            (None, 128)               25728     
                                                                 
 dense_15 (Dense)            (None, 1)                 129       
                                                                 
Total params: 4,389,857
Trainable params: 4,389,857
Non-trainable params: 0
_________________________________________________________________


2023-05-28 19:46:53.005974: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis' with dtype int32 and shape [1]
	 [[{{node gradients/ReverseV2_grad/ReverseV2/ReverseV2/axis}}]]
2023-05-28 19:46:53.067146: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-28 19:46:53.069564: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message):

In [58]:
# Used for preventing ovefitting
from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor="val_loss",patience=5,verbose=True)

In [59]:
history = model.fit(X_train_seq_padded, y_train,batch_size=BATCH_SIZE,epochs=1,
                    validation_data=(X_test_seq_padded, y_test),callbacks=[early_stop])

2023-05-28 19:47:00.152406: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-28 19:47:00.156742: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-28 19:47:00.159658: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

4803/4803 [==============================] - ETA: 0s - loss: 0.6265 - accuracy: 0.6465

2023-05-28 20:02:03.767575: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-28 20:02:03.773646: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-28 20:02:03.779342: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

4803/4803 [==============================] - 965s 200ms/step - loss: 0.6265 - accuracy: 0.6465 - val_loss: 0.6113 - val_accuracy: 0.6616


In [60]:
from sklearn.metrics import roc_auc_score
pred_train = model.predict(X_train_seq_padded)
pred_test = model.predict(X_test_seq_padded)
print('LSTM Recurrent Neural Network baseline: ' + str(roc_auc_score(y_train, pred_train)))
print('LSTM Recurrent Neural Network: ' + str(roc_auc_score(y_test, pred_test)))

2023-05-28 20:03:12.703478: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-05-28 20:03:12.706985: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-05-28 20:03:12.709634: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

5003/5003 [==============================] - 59s 12ms/step
LSTM Recurrent Neural Network baseline: 0.7397844717471033
LSTM Recurrent Neural Network: 0.7112987076776812


In [61]:
model.evaluate(X_test_seq_padded, y_test)

5003/5003 [==============================] - 63s 13ms/step - loss: 0.6113 - accuracy: 0.6616


[0.6113053560256958, 0.6615548729896545]